In [ ]:
from aiida import load_profile, common, orm 
load_profile();

from aiida.orm import QueryBuilder
import ipywidgets as widgets
from IPython.display import display
from widgets import query, misc
import aiidalab_widgets_base as awb
from settings import *
import pandas as pd

from IPython.core.display import display, HTML
display(HTML("""<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>"""))

# Inversion results

Remote directories of successful inversions. 

In [ ]:
qb = QueryBuilder()
qb.append(
            INVERSION,
            tag = 'inv',
            filters={"attributes.exit_status": 0},
            project=["id"],#ctime
        )
qb.append(orm.RemoteData, 
          with_incoming = 'inv',
          project = ['attributes.remote_path'])

for i in qb.all():
    print(i)